In [1]:
import os
from collections import defaultdict

# Define relative paths
INPUT_DIR = "../input"
OUTPUT_DIR = "../build"

# Input and output file paths
routers_list_path = os.path.join(INPUT_DIR, "sorted_routers_list.txt")
edgenumbers_path = os.path.join(INPUT_DIR, "edgenumbers.txt")
network_input_path = os.path.join(INPUT_DIR, "NetworkInput_BSNL.txt")
output_vertices_path = os.path.join(OUTPUT_DIR, "output_vertices.txt")
output_edges_path = os.path.join(OUTPUT_DIR, "output_edges.txt")

# Read sorted_routers_list.txt to map city IDs to names
city_names = {}
with open(routers_list_path, 'r') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            city_id, name = int(parts[0]), parts[1]
            city_names[city_id] = name

# Read edgenumbers.txt to extract coordinates
city_coords = {}
with open(edgenumbers_path, 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:  # Skip the first line (header)
        parts = line.strip().split()
        if len(parts) >= 9:
            v1, v2 = int(parts[1]), int(parts[5])
            lat1, lon1 = float(parts[3]), float(parts[4])
            lat2, lon2 = float(parts[7]), float(parts[8])
            if v1 not in city_coords:
                city_coords[v1] = (lat1, lon1)
            if v2 not in city_coords:
                city_coords[v2] = (lat2, lon2)

# Read NetworkInput_BSNL.txt to build adjacency list and degrees (directed graph)
degrees = defaultdict(int)  # Out-degree
adj_vertices = defaultdict(set)  # Outgoing neighbors
adj_edges = defaultdict(set)  # Outgoing edges (edgeno)
with open(network_input_path, 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines[1:], start=1):  # Skip first line (705 3164)
        parts = line.strip().split()
        if len(parts) >= 2:
            v1, v2 = int(parts[0]), int(parts[1])
            adj_vertices[v1].add(v2)  # Directed graph: add v2 as an outgoing neighbor of v1
            degrees[v1] += 1  # Increment out-degree of v1
            adj_edges[v1].add(i)  # Assign edgeno based on line number

# Write output_vertices.txt with S.NO, City_Id, City_Name, Degree, Adjacent_Vertices
with open(output_vertices_path, 'w') as f:
    header = f"{'S.NO':<6}  {'City_Id':<10}  {'City_Name':<20}  {'Degree':<8}  {'Adjacent_Vertices':<60}\n"
    f.write(header)
    for serial_no, city_id in enumerate(range(1, 706), start=1):
        name = city_names.get(city_id, 'UNKNOWN')
        degree = degrees[city_id]
        vertices = sorted(list(adj_vertices[city_id]))
        vertices_str = ' '.join(map(str, vertices)) if vertices else ''
        line = f"{serial_no:<6}  {city_id:<10}  {name:<20}  {degree:<8}  {vertices_str:<60}\n"
        f.write(line)

# Write output_edges.txt with S.NO, City_Id, City_Name, Degree, Adjacent_Edges
with open(output_edges_path, 'w') as f:
    header = f"{'S.NO':<6}  {'City_Id':<10}  {'City_Name':<20}  {'Degree':<8}  {'Adjacent_Edges':<60}\n"
    f.write(header)
    for serial_no, city_id in enumerate(range(1, 706), start=1):
        name = city_names.get(city_id, 'UNKNOWN')
        degree = degrees[city_id]
        edges = sorted(list(adj_edges[city_id]))
        edges_str = ' '.join(map(str, edges)) if edges else ''
        line = f"{serial_no:<6}  {city_id:<10}  {name:<20}  {degree:<8}  {edges_str:<60}\n"
        f.write(line)